In [1]:
#r "nuget: Newtonsoft.Json.Schema"
#r "nuget: dotenv.net"

Installed Packages dotenv.net, 3.2.1 Newtonsoft.Json.Schema, 4.0.1

In [2]:
using System;
using System.Net.Http;
using System.Threading.Tasks;
using System.Collections.Generic;
using System.Net.Http.Headers;
using System.Text.Json;
using System.Diagnostics;
using dotenv.net;

In [3]:
Console.WriteLine("Hello, .NET Interactive in Jupyter!");

Hello, .NET Interactive in Jupyter!


In [4]:
var assert = (bool condition, string message="") => { if (!condition) throw new Exception(message); };

# Sendungen API

In [6]:
using System;
using System.Net.Http;
using System.Net.Http.Headers;
using System.Threading.Tasks;
using Newtonsoft.Json;

// Define the AccessToken class if not already defined
public class AccessToken
{
    public string token_type { get; set; }
    public int expires_in { get; set; }
    public int ext_expires_in { get; set; }
    public string access_token { get; set; }
}

public class TokenService
{
    private readonly HttpClient _httpClient;

    public TokenService()
    {
        _httpClient = new HttpClient();
    }

    public async Task<AccessToken> GetAccessTokenAsync()
    {
        var url = Environment.GetEnvironmentVariable("MICROSOFT_AUTH_URL");
        var clientId = Environment.GetEnvironmentVariable("CLIENT_ID");
        var sendungenClientSecret = Environment.GetEnvironmentVariable("SENDUNGEN_CLIENT_SECRET");
        var sendungenScope = Environment.GetEnvironmentVariable("SENDUNGEN_SCOPE");

        var content = new FormUrlEncodedContent(new[]
        {
            new KeyValuePair<string, string>("client_id", clientId),
            new KeyValuePair<string, string>("client_secret", sendungenClientSecret),
            new KeyValuePair<string, string>("grant_type", "client_credentials"),
            new KeyValuePair<string, string>("scope", sendungenScope)
        });


        // Set headers to align with Postman configuration
        _httpClient.DefaultRequestHeaders.Accept.Add(new MediaTypeWithQualityHeaderValue("*/*"));
        _httpClient.DefaultRequestHeaders.UserAgent.ParseAdd("PostmanRuntime/7.42.0");
        _httpClient.DefaultRequestHeaders.AcceptEncoding.ParseAdd("gzip, deflate, br");
        _httpClient.DefaultRequestHeaders.Connection.ParseAdd("keep-alive");

        var response = await _httpClient.PostAsync(url, content);

        if (!response.IsSuccessStatusCode)
        {
            throw new Exception($"Error fetching token: {response.ReasonPhrase}");
        }

        var jsonResponse = await response.Content.ReadAsStringAsync();
        var accessToken = JsonConvert.DeserializeObject<AccessToken>(jsonResponse);

        return accessToken;
    }
}

In [7]:
var aa = await new TokenService().GetAccessTokenAsync();

Error: System.InvalidOperationException: An invalid request URI was provided. Either the request URI must be an absolute URI or BaseAddress must be set.
   at System.Net.Http.HttpClient.PrepareRequestMessage(HttpRequestMessage request)
   at System.Net.Http.HttpClient.SendAsync(HttpRequestMessage request, HttpCompletionOption completionOption, CancellationToken cancellationToken)
   at Submission#6.TokenService.GetAccessTokenAsync()
   at Submission#7.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

# Elfriede API

In [8]:
using System;
using System.Net.Http;
using System.Net.Http.Headers;
using System.Threading.Tasks;
using Newtonsoft.Json;

public class ElfriedeTokenService
{
    private readonly HttpClient _httpClient;

    public ElfriedeTokenService()
    {
        _httpClient = new HttpClient();
    }

    public async Task<AccessToken> GetElfriedeAccessTokenAsync()
    {

        // Fetch values from environment
        var url = Environment.GetEnvironmentVariable("MICROSOFT_AUTH_URL");
        var clientId = Environment.GetEnvironmentVariable("CLIENT_ID");
        var sendungenClientSecret = Environment.GetEnvironmentVariable("SENDUNGEN_CLIENT_SECRET");
        var elfriedeClientSecret = Environment.GetEnvironmentVariable("ELFRIEDE_CLIENT_SECRET");
        var sendungenScope = Environment.GetEnvironmentVariable("SENDUNGEN_SCOPE");
        var elfriedeScope = Environment.GetEnvironmentVariable("ELFRIEDE_SCOPE");

        var content = new FormUrlEncodedContent(new[]
        {
            new KeyValuePair<string, string>("client_id", clientId),
            new KeyValuePair<string, string>("client_secret", elfriedeClientSecret),
            new KeyValuePair<string, string>("grant_type", "client_credentials"),
            new KeyValuePair<string, string>("scope", elfriedeScope)
        });

        // Setting headers to align with the Postman configuration
        _httpClient.DefaultRequestHeaders.Accept.Add(new MediaTypeWithQualityHeaderValue("*/*"));
        _httpClient.DefaultRequestHeaders.UserAgent.ParseAdd("PostmanRuntime/7.42.0");
        _httpClient.DefaultRequestHeaders.AcceptEncoding.ParseAdd("gzip, deflate, br");
        _httpClient.DefaultRequestHeaders.Connection.ParseAdd("keep-alive");

        var response = await _httpClient.PostAsync(url, content);

        if (!response.IsSuccessStatusCode)
        {
            throw new Exception($"Error fetching token: {response.ReasonPhrase}");
        }

        var jsonResponse = await response.Content.ReadAsStringAsync();
        var accessToken = JsonConvert.DeserializeObject<AccessToken>(jsonResponse);

        return accessToken;
    }
}

public class AccessToken
{
    public string token_type { get; set; }
    public int expires_in { get; set; }
    public int ext_expires_in { get; set; }
    public string access_token { get; set; }
}

In [9]:
// var options = new JsonSerializerOptions
// {
//     PropertyNameCaseInsensitive = true, // Allows matching property names irrespective of their casing
//     PropertyNamingPolicy = JsonNamingPolicy.CamelCase // Configures the naming policy to handle camelCase
// };
var tokenService = new ElfriedeTokenService();
var accessToken = await tokenService.GetElfriedeAccessTokenAsync();

Console.WriteLine($"Token Type: {accessToken.token_type}");
Console.WriteLine($"Access Token: {accessToken.access_token}");

Error: System.InvalidOperationException: An invalid request URI was provided. Either the request URI must be an absolute URI or BaseAddress must be set.
   at System.Net.Http.HttpClient.PrepareRequestMessage(HttpRequestMessage request)
   at System.Net.Http.HttpClient.SendAsync(HttpRequestMessage request, HttpCompletionOption completionOption, CancellationToken cancellationToken)
   at Submission#8.ElfriedeTokenService.GetElfriedeAccessTokenAsync()
   at Submission#9.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [10]:
public static async Task<string> CallCreateUserEndpointAsync(AccessToken accessToken, string apiUri, int level, bool completeDoi)
{
    using var httpClient = new HttpClient();

    // Define the full endpoint URL with query parameters
    var apiEndpoint = $"{apiUri}/api/SsoUser?level={level}&completeDoi={completeDoi.ToString().ToLower()}";

    // Set the Authorization header with the acquired access token
    httpClient.DefaultRequestHeaders.Authorization = new AuthenticationHeaderValue(accessToken.token_type, accessToken.access_token);

    try
    {
        // Send the POST request to the specified endpoint
        var response = await httpClient.PostAsync(apiEndpoint, null);

        // Check if the response was successful
        if (response.IsSuccessStatusCode)
        {
            var apiResponseContent = await response.Content.ReadAsStringAsync();
            Console.WriteLine("API call succeeded. Response:");
            Console.WriteLine(apiResponseContent);
            return apiResponseContent;
        }
        else
        {
            Console.WriteLine($"API call failed with status code: {response.StatusCode}");
            var errorContent = await response.Content.ReadAsStringAsync();
            throw new Exception($"Error response: {errorContent}");
        }
    }
    catch (Exception ex)
    {
        throw new Exception($"An error occurred while calling the API: {ex.Message}");
    }
}

In [11]:
string apiUri = "https://elfriede-api.test.shared-k8s.postag.intern"; // Replace with your actual API URI
int level = 90; // Set the level value as required
bool completeDoi = false; // Set the completeDoi value as required

In [12]:
// Execute the function and display the response
var response = await CallCreateUserEndpointAsync(accessToken, apiUri, level, completeDoi);
Console.WriteLine(response);

Error: System.NullReferenceException: Object reference not set to an instance of an object.
   at Submission#10.<CallCreateUserEndpointAsync>d__1.MoveNext()
--- End of stack trace from previous location ---
   at Submission#12.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [13]:
// Define the API endpoint URL and access token obtained earlier
const string apiUri = "https://elfriede-api.test.shared-k8s.postag.intern"; // Replace with your specific endpoint if needed
const string apiEndpoint = $"{apiUri}/api/SsoUser?username=Sahra.Tuschmo@example.com"; // Example endpoint

In [14]:
// Function to call the API using the access token
async Task<string> CallApiAsync(AccessToken accessToken)
{
    using var httpClient = new HttpClient();

    // Set the Authorization header with the acquired access token
    httpClient.DefaultRequestHeaders.Authorization = new AuthenticationHeaderValue(accessToken.token_type, accessToken.access_token);

    try
    {
        // Send the GET request to the specified endpoint
        var response = await httpClient.GetAsync(apiEndpoint);

        // Check if the response was successful
        if (response.IsSuccessStatusCode)
        {
            var apiResponseContent = await response.Content.ReadAsStringAsync();
            Console.WriteLine("API call succeeded. Response:");
            Console.WriteLine(apiResponseContent);
            return apiResponseContent;
        }
        else
        {
            Console.WriteLine($"API call failed with status code: {response.StatusCode}");
            var errorContent = await response.Content.ReadAsStringAsync();
            throw new Exception($"Error response: {errorContent}");
        }
    }
    catch (Exception ex)
    {
        throw new Exception($"An error occurred while calling the API: {ex.Message}");
    }
}

// Execute the function with the acquired access token
var res = await CallApiAsync(accessToken: accessToken); // Replace "your-access-token-here" with the actual access token string from the previous step
res

Error: System.NullReferenceException: Object reference not set to an instance of an object.
   at Submission#14.<CallApiAsync>d__1.MoveNext()
--- End of stack trace from previous location ---
   at Submission#14.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)